In [50]:
import pandas as pd

data_dir = 'C:/ZhangLI/Codes/DataSet/个人违贷/official_data/'
train_pub = pd.read_csv(data_dir+'train_public.csv')
train_net = pd.read_csv(data_dir+'train_internet.csv')
test_pub = pd.read_csv(data_dir+'test_public.csv')

# 数据处理
train_net['isDefault'] = train_net['is_default']
common_feature = list(set(train_pub.columns).intersection(set(train_net.columns)))
import copy
train_pub_new = copy.deepcopy(train_pub)
train_net_new = copy.deepcopy(train_net)
c = ['class', 'employer_type', 'industry', 'work_year', 'issue_date', 'earlies_credit_mon']
common_feature = [i for i in common_feature if i not in c]
train_pub_new = train_pub_new[common_feature]
train_net_new = train_net_new[common_feature]
train_df = pd.concat([train_pub_new, train_net_new])
train_df.fillna(0, inplace=True)

In [51]:
# 查看样本比 并采样
# 正负样本比例
# 欠采样
def ratio_PN(dataframe=None, label=None):
    dic = train_df.isDefault.value_counts().to_dict()
    print(f'The number of positive and negative is: {dic}')
    print(f'0:1：{dic[0]/dic[1]}')
# 重采样
# Shuffle the Dataset.
def sample_train_data(dataframe=None, label=None, ratio=1):
    # rate: the rate of 0:1
    shuffled_df = dataframe.sample(frac=1,random_state=4)
    # 1
    positive_df = shuffled_df.loc[shuffled_df[label] == 1]
    n = positive_df.shape[0] * ratio
    # 0
    negtive_df=shuffled_df.loc[shuffled_df[label]== 0].sample(n=n,random_state=42)
    # concat
    normalized_df = pd.concat([positive_df, negtive_df])
    return normalized_df
    
# normalized_df = sample_train_data(train_df, 'isDefault', 2)
# 过采样 (Synthetic Minority Over-sampling Technique)
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority', random_state=7)

# Fit the model to generate the data.
oversampled_trainX,oversampled_trainY=sm.fit_resample(train_df.drop('isDefault', axis=1), train_df['isDefault'])

oversampled_train=pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis=1)

oversampled_train.columns = normalized_df.columns

ValueError: Length mismatch: Expected axis has 30 elements, new values have 36 elements

In [48]:
train_df.isna().sum()

early_return_amount             0
pub_dero_bankrup              382
loan_id                         0
monthly_payment                 0
year_of_loan                    0
f1                          66269
f2                          38296
del_in_18month                  0
f3                          38297
policy_code                     0
scoring_high                    0
scoring_low                     0
f0                          38296
total_loan                      0
initial_list_status             0
early_return                    0
house_exist                     0
interest                        0
early_return_amount_3mon        0
recircle_u                    489
recircle_b                      0
isDefault                       0
f4                          38296
user_id                         0
debt_loan_ratio               230
censor_status                   0
post_code                       1
title                           1
region                          0
use           

In [52]:

from imblearn.ensemble import BalancedBaggingClassifier

from sklearn.tree import DecisionTreeClassifier

#Create an object of the classifier.

bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),

                                sampling_strategy='auto',

                                replacement=False,

                                random_state=0)

y_train = train_df['isDefault']

X_train = train_df.drop(['isDefault'], axis=1, inplace=False)

#Train the classifier.

bbc.fit(X_train, y_train)

preds = bbc.predict(X_train)

In [53]:
preds

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)